# Deep Learning

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from models import Six
import torch
from torch.utils.data import Dataset, DataLoader
import onnxruntime as ort

/home/hoangphuc/miniconda3/envs/gene/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "checkpoints/ckp_six.pt"
checkpoint_bin = "checkpoints/ckp_six_last.pt"
model = Six()
checkpoint_dict = torch.load(checkpoint, map_location='cpu')
print("Load model control or diseases")
model.load_state_dict(checkpoint_dict['net'])
model.to('cuda:0')
print("f1 score: ", checkpoint_dict['f1'])

input_names = ["gene"]
output_names = ["result"]
dummy = torch.randn(1,8630, device='cuda')
model.eval()
torch.onnx.export(model, dummy, "checkpoints/Six.onnx", input_names=input_names, output_names=output_names, verbose=True)

Load model control or diseases
f1 score:  0.850108225108225
Exported graph: graph(%gene : Float(1, 8630, strides=[8630, 1], requires_grad=0, device=cuda:0),
      %fc1.weight : Float(8192, 8630, strides=[8630, 1], requires_grad=1, device=cuda:0),
      %fc1.bias : Float(8192, strides=[1], requires_grad=1, device=cuda:0),
      %fc2.weight : Float(2048, 8192, strides=[8192, 1], requires_grad=1, device=cuda:0),
      %fc2.bias : Float(2048, strides=[1], requires_grad=1, device=cuda:0),
      %fc3.weight : Float(128, 2048, strides=[2048, 1], requires_grad=1, device=cuda:0),
      %fc3.bias : Float(128, strides=[1], requires_grad=1, device=cuda:0),
      %layer_out.weight : Float(6, 128, strides=[128, 1], requires_grad=1, device=cuda:0),
      %layer_out.bias : Float(6, strides=[1], requires_grad=1, device=cuda:0),
      %batchnorm1.weight : Float(8192, strides=[1], requires_grad=1, device=cuda:0),
      %batchnorm1.bias : Float(8192, strides=[1], requires_grad=1, device=cuda:0),
      %ba

In [3]:
ckp = "checkpoints/Six.onnx"

class2idx = {
    "CRC":0,
    "Control":1,
    "Lung":2,
    "Gastric":3,
    "Liver":4,
    "Breast":5
}

idx2class = {v: k for k, v in class2idx.items()}

testloader = []

with open('data/val_six.npy', 'rb') as f:
    x_test0 = np.load(f)
with open('data/val_six_target.npy', 'rb') as f:
    y_test0 = np.load(f)

with open('data/test_six.npy', 'rb') as f:
    x_test1 = np.load(f)
with open('data/test_six_target.npy', 'rb') as f:
    y_test1 = np.load(f)

x_test = np.concatenate([x_test0, x_test1])
y_test = np.concatenate([y_test0, y_test1])
randomize = np.arange(len(x_test))
np.random.shuffle(randomize)
x_test = x_test[randomize]
y_test = y_test[randomize]

x_test = np.array(x_test, np.float32)
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


test_dataset = ClassifierDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).long())

testloader = DataLoader(dataset=test_dataset, batch_size=1)

cuda=True
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if cuda else ['CPUExecutionProvider']
session = ort.InferenceSession(ckp, providers=providers)

/home/hoangphuc/miniconda3/envs/gene/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


In [4]:
inname = [i.name for i in session.get_inputs()]
inname

['gene']

In [5]:
outname = [i.name for i in session.get_outputs()]
outname

['result']

In [6]:
idx = 0
x = np.expand_dims(x_test[idx], axis=0)
print(x)
print(y_test[idx])
print("result:", idx2class[y_test[idx]])

[[0.28597075 0.6151032  0.6775692  ... 0.18518262 0.30795324 0.1246049 ]]
4
result: Liver


In [7]:
x = np.expand_dims(x_test[idx], axis=0)

out = session.run(outname,{'gene':x})
result = np.argmax(out)
print("Result: ",idx2class[result])


Result:  Liver


# Machine Learning

In [8]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

In [9]:
# load the model from disk
filename = 'checkpoints/ml_svm_six.sav'
svm_ = pickle.load(open(filename, 'rb'))
filename = 'checkpoints/ml_rf_six.sav'
rf = pickle.load(open(filename, 'rb'))
filename = 'checkpoints/ml_knn_six.sav'
knn = pickle.load(open(filename, 'rb'))

class2idx = {
    "CRC":0,
    "Control":1,
    "Lung":2,
    "Gastric":3,
    "Liver":4,
    "Breast":5
}

idx2class = {v: k for k, v in class2idx.items()}

testloader = []

with open('data/val_six.npy', 'rb') as f:
    x_test0 = np.load(f)
with open('data/val_six_target.npy', 'rb') as f:
    y_test0 = np.load(f)

with open('data/test_six.npy', 'rb') as f:
    x_test1 = np.load(f)
with open('data/test_six_target.npy', 'rb') as f:
    y_test1 = np.load(f)

x_test = np.concatenate([x_test0, x_test1])
y_test = np.concatenate([y_test0, y_test1])

randomize = np.arange(len(x_test))
np.random.shuffle(randomize)
x_test = x_test[randomize]
y_test = y_test[randomize]

In [42]:
idx = 0
x = np.expand_dims(x_test[idx], axis=0)
print(x)
print(y_test[idx])
print("result:", idx2class[y_test[idx]])

[[0.37333759 0.52708523 0.55567462 ... 0.75800663 0.57523247 0.67382827]]
4
result: Liver


In [43]:
x = np.expand_dims(x_test[idx], axis=0)

result = svm_.predict(x)[0]
print("Result: ",idx2class[result])

Result:  CRC


In [44]:
x = np.expand_dims(x_test[idx], axis=0)

result = rf.predict(x)[0]
print("Result: ",idx2class[result])

Result:  CRC


In [45]:
x = np.expand_dims(x_test[idx], axis=0)

result = knn.predict(x)[0]
print("Result: ",idx2class[result])

Result:  CRC
